In [ ]:
SCOPETYPE = 'CWNANO'
PLATFORM = 'CWNANO'

INFO: Found ChipWhisperer😍


In [ ]:
%run "../setup/Setup_Generic.ipynb"

INFO: Found ChipWhisperer😍


In [6]:
cw.program_target(scope, prog, "ghostBlood-{}.hex".format(PLATFORM))

Detected known STMF32: STM32F04xxx
Extended erase (0x44), this can take ten seconds or more
Attempting to program 13559 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 13559 bytes


## Capture and Analyze

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import string
from sklearn.cluster import KMeans
from scipy.stats import pearsonr

# Number of power traces
num_traces = 256*196

# Define an array to store all the power traces
power_traces = []
    
num = 1
pos=1
key = b'\x00'*16 

power_traces = []
plaintexts = []
passwords = []
count = 0
for k in range (0, 500000):
    target.simpleserial_write('k', key)
    resp = target.simpleserial_read('r', 8)
    if resp is not None:
        password = list(resp)
        
        tx = b'\x01\x00\x07\x09\x0D\x12'

        # Arm the Chipwhisper
        scope.arm()

        target.simpleserial_write('s', bytearray(tx))

        # Capture the power trace
        ret = scope.capture()
        if ret:
            print('Timeout happened during acquisition')

        resp = target.simpleserial_read('r', 1)
        if resp is not None:
            print("a: ", resp)

        # Read the power trace from the ChipWhisperer Nano
        power_trace = scope.get_last_trace()

        # Append the power trace to the list of power traces
        power_traces.append(power_trace)
        passwords.append(password)
        print("Password: ", password)

        if (((k+1) % 50000) == 0 and k>0):
            np.savez('/home/dakshina/projects/csaw_esc_2025/ss2/rand_with_pass_500k/power_traces_and_plaintexts_{}.npz'.format(k//50000), power_traces=np.array(power_traces), passwords=np.array(passwords))
            power_traces = []
            passwords = []

In [16]:
tx = b'\x00\x00\x00\x00\x00\x00'
target.simpleserial_write('s', tx)
response = target.simpleserial_read('r', 1)
if response is not None:
    print(response)

key = b'\x90\x18\x89\xf6\x4c\x39\x31\x8a\x92\x4d\x72\xed\x23\xdc\x23\x01'
target.simpleserial_write('d', key)
response = target.simpleserial_read('r', 21)
if response is not None:
    print(response)

queries = b'\x00'
target.simpleserial_write('q', queries)
response = target.simpleserial_read('r', 4)
if response is not None:
    print(f"Number of Queries (cumulative): {int.from_bytes(response, byteorder='little', signed=False)}")

bytearray(b'\x01')
bytearray(b'ESC{\xca\xbc\n\x80a,\xdf\x1d7J\xd0\xc2\x08v\x0f|}')
Number of Queries (cumulative): 18
